<a href="https://colab.research.google.com/github/raphamonn/Decision-Tree-with-PokemonData/blob/main/pre_processing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Data Preprocessing Template

## Importando as bibliotecas

In [47]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [48]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os


## Importando o Dataset


In [55]:
dataset = pd.read_csv("/content/drive/My Drive/Colab Notebooks/Machine Learning A-Z (Codes and Datasets)/Part 1 - Data Preprocessing/Section 2 -------------------- Part 1 - Data Preprocessing --------------------/Python/Data.csv")
display(dataset.head())

,Country,Age,Salary,Purchased
0,France,44.0,72000.0,No
1,Spain,27.0,48000.0,Yes
2,Germany,30.0,54000.0,No
3,Spain,38.0,61000.0,No
4,Germany,40.0,NaN,Yes


In [95]:
# Criando a matriz de variáveis independentes, que serão utilizadas como preditores
# Em python, um range inclui o limite inferior, porém exclui o limite superior, então o 0:-1 vai do primeiro index que é o 0, mas somente até o penúltimo item para esse range
# Logo x tem todas as linhas, e todas colunas exceto a última
x = dataset.iloc[0:, 0:-1].values

#Criando o vetor de variáveis dependentes, que são as variáveis que queremos prever com nosso modelo
# Como não estamos usando um range abaixo para colunas, o -1 representa o último item de fato
y = dataset.iloc[0:, -1].values

#Essa construção é para que o modelo de machine learning que vamos construir já receba as estruturas da forma que ele precisa

## Pré Processamento dos dados

## Cuidado dos dados nulos

Temos algumas formas de lidar com dados nulos, quando temos um grande conjunto de dados, e 1% deles é nulo, tudo bem removê-los, não alterará muito o funcionamento do modelo, mas quando temos um volume pequeno ou uma porcentagem significativa de nulos, precisamos tratar de forma diferente

In [59]:
x

array([['France', 44.0, 72000.0],
       ['Spain', 27.0, 48000.0],
       ['Germany', 30.0, 54000.0],
       ['Spain', 38.0, 61000.0],
       ['Germany', 40.0, nan],
       ['France', 35.0, 58000.0],
       ['Spain', nan, 52000.0],
       ['France', 48.0, 79000.0],
       ['Germany', 50.0, 83000.0],
       ['France', 37.0, 67000.0]], dtype=object)

In [60]:
# Verificando os dados nulos
dataset.isnull().sum()

,0
Country,0
Age,1
Salary,1
Purchased,0


A outra forma de tratar, é substituir os dados faltantes pela média dos valores na coluna em que o dado está nulo, usando o simpleimputer do Sklearn, primeiro importaremos a classe e depois criaremos um objeto do simpleimputer

In [61]:
from sklearn.impute import SimpleImputer
# definimos os valores que o imputer irá atuar, que serão os nulos, e a
# estratégia diz que substituiremos pela média, poderia ser a mediana por exemplo, ou a moda
imputer = SimpleImputer(missing_values=np.nan, strategy='mean')
# classes tem métodos que são parecidos com funçoes, e um dos métodos que o imputer tem, é exatamente o
#fit que verificar qual dado que será utilizado, meio que mapeando o que será aplicado
imputer.fit(x[:,1:3])
#posteriormente usaremos o transform para aplicar a transformação
x[:, 1:3] = imputer.transform(x[:,1:3])

In [62]:
x[:, 1:3]

array([[44.0, 72000.0],
       [27.0, 48000.0],
       [30.0, 54000.0],
       [38.0, 61000.0],
       [40.0, 63777.77777777778],
       [35.0, 58000.0],
       [38.77777777777778, 52000.0],
       [48.0, 79000.0],
       [50.0, 83000.0],
       [37.0, 67000.0]], dtype=object)

### Teste usando pandas e dataset ao invés de matriz numérica (bem melhor pra mim)

In [63]:
xv1 = dataset.iloc[:,0:-1].copy()
yv1 = dataset.iloc[:,-1].copy()

In [64]:
imputerv1 = SimpleImputer(missing_values=np.nan, strategy='mean')
imputerv1.fit(xv1.iloc[:,1:].values)
xv1.iloc[:,1:] = imputerv1.transform(xv1.iloc[:,1:].values)

## Codificando os dados categóricos

Os nossos países poderiam ser codificados como 1 2 e 3, mas como um futuro modelo de machine learning poderia interpretar uma ordem hierarquica, ou simplesmente que um é maior ou menor que o outro, e isso não é o que queremos, poderia ser, mas não é no momento, não há relação ordinal entre os países

In [65]:
dataset.head(5)

,Country,Age,Salary,Purchased
0,France,44.0,72000.0,No
1,Spain,27.0,48000.0,Yes
2,Germany,30.0,54000.0,No
3,Spain,38.0,61000.0,No
4,Germany,40.0,NaN,Yes


### Codificando as variáveis independentes
*São as variáveis preditoras cujo uso é para prever algo, ou seja, a idade, o salário, os objetos que usaremos como ferramentas preditoras*

**Para isso, utilizaremos One Hot Encoding (parecido com Get_dummies)**

IMPORTANTE: tive problemas testando no dataset porque usei somente o transform após aplicar o fit_transform, e como ele é fitado para 3 features que são as que estão em x, ele tem problemas para aplicar isso em um dataset com 4 colunas se for usar o transform, exigindo o uso de fit denovo ou um fit_transform

In [66]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

# Lembre, para criar um objeto de uma classe, precisa chamar a
# própria classe passando parâmetros tal qual uma função
ct = ColumnTransformer(
    transformers=[('encoder', OneHotEncoder(), [0])],
    remainder='passthrough'
    )

x= np.array(ct.fit_transform(x))

In [67]:
print(x)

[[1.0 0.0 0.0 44.0 72000.0]
 [0.0 0.0 1.0 27.0 48000.0]
 [0.0 1.0 0.0 30.0 54000.0]
 [0.0 0.0 1.0 38.0 61000.0]
 [0.0 1.0 0.0 40.0 63777.77777777778]
 [1.0 0.0 0.0 35.0 58000.0]
 [0.0 0.0 1.0 38.77777777777778 52000.0]
 [1.0 0.0 0.0 48.0 79000.0]
 [0.0 1.0 0.0 50.0 83000.0]
 [1.0 0.0 0.0 37.0 67000.0]]


#### Exemplo usando get_dummies ao invés de One hot encoding no próprio dataframe

In [68]:
dummies = pd.get_dummies(dataset['Country'], dtype=int)
df1 = pd.concat([dummies,dataset.iloc[:,1:]], axis=1 )
np.array(df1.iloc[:,0:-1].values)


array([[1.0e+00, 0.0e+00, 0.0e+00, 4.4e+01, 7.2e+04],
       [0.0e+00, 0.0e+00, 1.0e+00, 2.7e+01, 4.8e+04],
       [0.0e+00, 1.0e+00, 0.0e+00, 3.0e+01, 5.4e+04],
       [0.0e+00, 0.0e+00, 1.0e+00, 3.8e+01, 6.1e+04],
       [0.0e+00, 1.0e+00, 0.0e+00, 4.0e+01,     nan],
       [1.0e+00, 0.0e+00, 0.0e+00, 3.5e+01, 5.8e+04],
       [0.0e+00, 0.0e+00, 1.0e+00,     nan, 5.2e+04],
       [1.0e+00, 0.0e+00, 0.0e+00, 4.8e+01, 7.9e+04],
       [0.0e+00, 1.0e+00, 0.0e+00, 5.0e+01, 8.3e+04],
       [1.0e+00, 0.0e+00, 0.0e+00, 3.7e+01, 6.7e+04]])

### Codificando as variáveis dependentes
*A variável dependente é aquela cujo o resultado deve ser previsto, ou seja, o valor de um carro, o tipo de uma amostra, coisas assim, é o nosso target*

In [69]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
y = le.fit_transform(y)

## Separando o dataset em Treino e teste
Aqui teremos a separação que definirá a porcentagem de dados que terá o treino e o teste, normalmente algo entre 80-02 ou 70-30, depende do volume de dados e do contexto, no caso, usaremos o random_state=1 para fixar a seed específica e tornar o experimento reprodutível

In [70]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(x,y, train_size=0.8, random_state=1)

In [71]:
print(X_train)
print('____________________________')
print(y_train)

[[0.0 0.0 1.0 38.77777777777778 52000.0]
 [0.0 1.0 0.0 40.0 63777.77777777778]
 [1.0 0.0 0.0 44.0 72000.0]
 [0.0 0.0 1.0 38.0 61000.0]
 [0.0 0.0 1.0 27.0 48000.0]
 [1.0 0.0 0.0 48.0 79000.0]
 [0.0 1.0 0.0 50.0 83000.0]
 [1.0 0.0 0.0 35.0 58000.0]]
____________________________
[0 1 0 0 1 1 0 1]


In [72]:
print(X_test)
print('____________________________')
print(y_test)

[[0.0 1.0 0.0 30.0 54000.0]
 [1.0 0.0 0.0 37.0 67000.0]]
____________________________
[0 1]


## Feature Scaling

Quando temos que calcular uma distância e temos variáveis de escalas muito distintas (exemplo: Peso e salário ) , precisamos tratá-las com normalização ou standardization, precisamos aplicar a divisão dos dados em treino e teste primeiro para evitar data leakage, isso é porque os dados do teste não podem ser transformados a esse nível, já que eles em tese são dados que nunca vimos, não podem estar presentes

In [81]:
from sklearn.preprocessing import StandardScaler

# Nós precisamos aplicar o StandarScaler nos Dummies? no caso, nas colunas criadas pelo OneHotEncoder?
# Não, não precisamos criar, nós precisamos do Standardization para que nossas features fiquem no range de -3,+3 ou 0,1 dependendo da técnica utilizada

sc = StandardScaler()

sc.fit(X_train[:,3:])

StandardScaler()

In [87]:
X_train[:,3:] = sc.transform(X_train[:,3:])

In [88]:
X_train

array([[0.0, 0.0, 1.0, -0.19159184384578545, -1.0781259408412425],
       [0.0, 1.0, 0.0, -0.014117293757057777, -0.07013167641635372],
       [1.0, 0.0, 0.0, 0.566708506533324, 0.633562432710455],
       [0.0, 0.0, 1.0, -0.30453019390224867, -0.30786617274297867],
       [0.0, 0.0, 1.0, -1.9018011447007988, -1.420463615551582],
       [1.0, 0.0, 0.0, 1.1475343068237058, 1.232653363453549],
       [0.0, 1.0, 0.0, 1.4379472069688968, 1.5749910381638885],
       [1.0, 0.0, 0.0, -0.7401495441200351, -0.5646194287757332]],
      dtype=object)

Aqui aplicaremos o MESMO sc que foi gerado na célula anterior, porque nós queremos aplicar o que foi fittado sem gerar um data leak no modelo, tudo deve ser treinado no modelo de treino, e não no de teste

In [90]:
X_test[:,3:] = sc.transform(X_test[:,3:])

In [91]:
X_train

array([[0.0, 0.0, 1.0, -0.19159184384578545, -1.0781259408412425],
       [0.0, 1.0, 0.0, -0.014117293757057777, -0.07013167641635372],
       [1.0, 0.0, 0.0, 0.566708506533324, 0.633562432710455],
       [0.0, 0.0, 1.0, -0.30453019390224867, -0.30786617274297867],
       [0.0, 0.0, 1.0, -1.9018011447007988, -1.420463615551582],
       [1.0, 0.0, 0.0, 1.1475343068237058, 1.232653363453549],
       [0.0, 1.0, 0.0, 1.4379472069688968, 1.5749910381638885],
       [1.0, 0.0, 0.0, -0.7401495441200351, -0.5646194287757332]],
      dtype=object)

In [92]:
X_test

array([[0.0, 1.0, 0.0, -1.4661817944830124, -0.9069571034860727],
       [1.0, 0.0, 0.0, -0.44973664397484414, 0.2056403393225306]],
      dtype=object)

In [93]:
y_train

array([0, 1, 0, 0, 1, 1, 0, 1])

In [94]:
y_test

array([0, 1])